In [119]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
import numpy as np
import cv2
from skimage import io
import requests
import pandas as pd
import feature_extractor , config_utils


fe=feature_extractor.FeatureExtractor()

553467096/553467096 [==============================] - 8s 0us/step


In [116]:

df=pd.read_csv('/content/drive/MyDrive/SearchEngineData/perfect/perfect90k.csv',usecols=['ImageID','OriginalURL','Tags', 'Author','Title'])
df.set_index("ImageID", inplace=True)



In [170]:
extracted_features={}
for url in df['OriginalURL']:
    try:
        response = requests.get(url)     
        extracted_features[url]=fe.get_features_from_link(response)
    except Exception as e:
        print(f"Error processing URL {url}: {e}")

# Combine the extracted features into a NumPy array


1/1 [==============================] - 1s 1s/step
Error processing URL https://farm5.staticflickr.com/5128/5301868579_f042b35323_o.jpg: HTTP Error 410: Gone
1/1 [==============================] - 1s 700ms/step


In [171]:
extracted_features

{'https://farm3.staticflickr.com/5310/5898076654_51085e157c_o.jpg': array([0.       , 4.3512473, 0.       , ..., 0.       , 0.       ,
        0.       ], dtype=float32),
 'https://c6.staticflickr.com/1/223/495658260_8947fc0be8_o.jpg': array([0.       , 0.       , 0.       , ..., 0.       , 2.1858375,
        0.       ], dtype=float32),
 'https://c1.staticflickr.com/1/67/197493648_628a7cb2ee_o.jpg': array([0.      , 0.      , 0.      , ..., 0.      , 2.144457, 0.      ],
       dtype=float32),
 'https://c2.staticflickr.com/8/7237/6883453992_4f68f3cca7_o.jpg': array([0.       , 1.015573 , 0.       , ..., 3.4672844, 0.       ,
        0.       ], dtype=float32),
 'https://c5.staticflickr.com/8/7153/6698283389_ecab2a15e8_o.jpg': array([ 0.      , 16.753206,  0.      , ...,  0.      ,  0.      ,
         0.      ], dtype=float32),
 'https://farm4.staticflickr.com/8016/7623550354_00cd2d195f_o.jpg': array([0.        , 0.        , 0.        , ..., 0.        , 0.04798944,
        0.        ], 

In [187]:
def euclidean_distance(vector1, vector2):
    """
    Calculate the Euclidean distance between two vectors.

    Parameters:
    - vector1, vector2: NumPy arrays or lists representing the two vectors.

    Returns:
    - The Euclidean distance between the two vectors.
    """
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    distance = np.linalg.norm(vector1 - vector2)
    return distance

In [204]:
url="https://c1.staticflickr.com/9/8671/15850875545_23d03bec40_o.jpg"
response = requests.get(url)

img = io.imread(url)
img = cv2.resize(img,(224, 224),interpolation=cv2.INTER_AREA)  # VGG must take a 224x224 img as an input
x = image.img_to_array(img)  # To np.array. Height x Width x Channel. dtype=float32
x = np.expand_dims(x, axis=0)  # (H, W, C)->(1, H, W, C), where the first elem is the number of img
x = preprocess_input(x)  # Subtracting avg values for each pixel
feature = model.predict(x)
reduced_image_features = pca.transform(feature)

min_distance = float('inf')  # Initialize with a large value
closest_feature_vector = None
for key  in reduced_dict.keys():  # Iterate over all feature vectors in your dataset
    distance = euclidean_distance(reduced_dict[key], reduced_image_features)  # Implement this function

    if distance < min_distance:
        min_distance = distance
        closest_feature_key= key


1/1 [==============================] - 1s 815ms/step


In [206]:
closest_feature_key

'https://c1.staticflickr.com/9/8671/15850875545_23d03bec40_o.jpg'

In [205]:
reduced_dict[closest_feature_key]

array([[-1.55322294e+01, -2.90414219e+01, -1.05942736e+01,
         1.41809559e+01,  2.06612263e+01,  6.59171295e+00,
        -1.03318481e+01, -4.49137402e+00, -5.77386379e+00,
        -2.77629089e+01,  9.57240486e+00,  7.54399014e+00,
        -9.81125832e+00,  1.89098911e+01, -1.08788939e+01,
         3.56242508e-01,  4.68040514e+00, -1.81313837e+00,
        -5.82387829e+00, -7.87701607e+00, -7.13665056e+00,
        -4.83927059e+00,  5.02675772e-02, -1.28178463e+01,
         1.32536907e+01, -2.58613014e+00,  2.44120598e+00,
        -5.34234643e-01,  1.51165116e+00, -9.71129799e+00,
         2.44550152e+01,  4.73913193e+00, -2.43307457e+01,
        -9.53657913e+00,  9.75054169e+00, -3.69755030e+00,
        -1.57235069e+01, -3.97020030e+00, -2.88981700e+00,
        -5.26540947e+00,  7.32151031e+00, -2.17099609e+01,
        -1.15771370e+01, -1.04061384e+01,  2.10957851e+01,
        -4.87545204e+00, -7.96586692e-01, -2.45016861e+00,
         9.40666676e+00, -5.07375121e-01,  7.35396481e+0

In [202]:
reduced_dict={}
for k in extracted_features.keys():
  reduced_dict[k]=pca.transform(extracted_features[k].reshape(1, -1))

In [128]:
from sklearn.decomposition import PCA

# Initialize the PCA model with the desired number of components (785)
n_components = 785
pca = PCA(n_components=n_components)

# Fit the PCA model to your 4096-dimensional features
pca.fit(extracted_features)

# Transform the features to the reduced dimension
reduced_dict={}
for k in extracted_features.keys():
  reduced_dict[k]=pca.transform(extracted_features[k].reshape(1, -1))

In [132]:
import pickle

# Assuming you have a PCA model named 'pca'
# and you want to save it to a file named 'pca_model.pkl'

# Save the PCA model to a file
with open(config_utils.pca_path, 'wb') as file:
    pickle.dump(pca, file)